In [1]:
from bs4 import BeautifulSoup
import requests 
import time
import re

all_indexes = [str(x) for x in range(1,28)]
all_indexes[0] = "0001"

all_universities_data = []

p = re.compile("^\/.*htm$")

for num_index in all_indexes: 
    page = requests.get("http://www.4icu.org/reviews/index%s.htm" % num_index)
    time.sleep(1) # pause 1 sec between gets
    
    soup = BeautifulSoup(page.text)

    universities_table = soup.find_all("tr")

    for university in universities_table:
        if university.img:
            uni_img_alt = university.img.get("alt").split(">")
            if len(uni_img_alt)>1:
                one_uni  = {"name":uni_img_alt[1].strip(), "country":uni_img_alt[0].strip()}            
                tmp_url = p.findall(university.find_all("a")[1].get("href"))

                if tmp_url:
                    
                    # this is in order to actually get university URL
                    # there are 10,000+ unis, this is not very relevant at the moment,
                    # so index will be the site uni url
                    #print university.find_all("a")

                    #page2 = requests.get("http://www.4icu.org" + tmp_url[0])
                    #soup2 = BeautifulSoup(page2.text)
                    #one_uni["url"]= soup2.find(class_="span_1_of_2").a.get("href")
                    one_uni["url"]= "http://www.4icu.org" + tmp_url[0]
                    
                if one_uni not in all_universities_data and "url" in one_uni.keys():
                    all_universities_data.append(one_uni)
                
print len(all_universities_data)

10881


In [2]:
# insert to db

from pymongo import MongoClient

connection = MongoClient('localhost', 27017)
db = connection.journals
collection = db.universities

collection.insert(all_universities_data)

connection.close()

In [4]:
# after insert, indexed by url (asc)
import pymongo
from pymongo import MongoClient

connection = MongoClient('localhost', 27017)
db = connection.journals
collection = db.universities
collection.create_index("url", pymongo.ASCENDING)

connection.close()